In [1]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, \
    Flatten, Lambda, LSTM, RepeatVector, TimeDistributed, Reshape, \
    Conv2D, MaxPooling2D, BatchNormalization, ConvLSTM2D, Bidirectional
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

/usr/share/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = h5py.File("../dataset/data_sample.hdf5", "r")
print("data keys: " + str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [3]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = " + str(images.shape))
labels = data["labels"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [4]:
def scale_images(images):
    t_images = np.transpose(images, (0,3,1,2))
    rt_images = t_images.reshape(72000*48, 21, 21)
    max_per_img = np.max(rt_images.reshape(-1, 21*21), axis=1, keepdims=1)
    scaled_images = rt_images.reshape(-1, 21*21) / max_per_img
    scaled_images = scaled_images.reshape(-1, 21, 21).reshape(-1, 48, 21, 21)
    return scaled_images

In [5]:
def txt2digit(labels):
    dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
    labels_digit = np.array([dic[i] for i in labels])
    return labels_digit

In [6]:
def build_dataset(images, labels, seq2seq=False):
    
    train_indices = np.random.choice(np.arange(images.shape[0]), int( 0.7 * images.shape[0]))
    val_indices = list(set(np.arange(images.shape[0])) - set(train_indices))
    
    x = images
    x = np.expand_dims(x, len(x.shape))
    y = to_categorical(txt2digit(labels))
    
    if seq2seq:
        y = np.repeat(y, 48).reshape(-1, 48, 6)
    
    return x, y

In [7]:
x, y = build_dataset(scale_images(images), labels)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=209, stratify=y)
print(x_train.shape,'\n',y_train.shape)

(57600, 48, 21, 21, 1) 
 (57600, 6)


In [8]:
def init_model3(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48,
               bidir_mode='concat'):
    
    input_shape = (timestep, cnn_input_dim, cnn_input_dim, 1, )
    
    model = Sequential()
    
    model.add(TimeDistributed(Conv2D(48, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), input_shape=input_shape))
    model.add(Dropout(cnn_dropout))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    
    model.add(TimeDistributed(Conv2D(24, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(Dropout(cnn_dropout))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))

    model.add(TimeDistributed(Conv2D(12, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(Dropout(cnn_dropout))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    
    model.add(TimeDistributed(Flatten()))

    model.add(TimeDistributed(Dense(128, activation='relu')))
    model.add(Dropout(cnn_dropout))
    
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True), \
                          input_shape=(timestep, cnn_output_dim), merge_mode=bidir_mode))
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout), merge_mode=bidir_mode))
    
    model.add(Dense(6, activation='softmax'))
    
    return model

In [9]:
### Dropout for loop
import json

# dp_list = [0.0,0.1,0.3,0.5]
batch_size = 80
epochs = 100
opt = keras.optimizers.Adam(lr=5*1e-4)
history_ls = []
dropout = 0.3
model3_drop = init_model3(bidir_mode='ave',cnn_dropout=dropout,rnn_dropout=dropout)
model3_drop.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
history = model3_drop.fit(x_train[::5], y_train[::5],
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)
# history_ls.append(history)
filename = 'history_do_3.json'
with open(filename, 'w') as outfile:
    json.dump(history.history, outfile)

Train on 11520 samples, validate on 14400 samples
Epoch 1/100
11520/11520 [==============================] - 145s 13ms/step - loss: 1.6095 - acc: 0.2608 - val_loss: 1.7720 - val_acc: 0.1729
Epoch 2/100
11520/11520 [==============================] - 135s 12ms/step - loss: 1.3537 - acc: 0.3759 - val_loss: 1.3569 - val_acc: 0.4405
Epoch 3/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.1630 - acc: 0.4317 - val_loss: 1.3174 - val_acc: 0.3485
Epoch 4/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.1196 - acc: 0.4408 - val_loss: 2.7417 - val_acc: 0.1929
Epoch 5/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.1100 - acc: 0.4529 - val_loss: 2.7797 - val_acc: 0.1917
Epoch 6/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.5489 - acc: 0.3217 - val_loss: 1.5862 - val_acc: 0.2530
Epoch 7/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.2601 - acc: 0.4028 - val_l

Epoch 59/100
11520/11520 [==============================] - 135s 12ms/step - loss: 0.8173 - acc: 0.5850 - val_loss: 1.3988 - val_acc: 0.5147
Epoch 60/100
11520/11520 [==============================] - 135s 12ms/step - loss: 0.7743 - acc: 0.6055 - val_loss: 1.4126 - val_acc: 0.5210
Epoch 61/100
11520/11520 [==============================] - 135s 12ms/step - loss: 0.7757 - acc: 0.6019 - val_loss: 1.5236 - val_acc: 0.5225
Epoch 62/100
11520/11520 [==============================] - 135s 12ms/step - loss: 0.7509 - acc: 0.6084 - val_loss: 1.6381 - val_acc: 0.4665
Epoch 63/100
11520/11520 [==============================] - 135s 12ms/step - loss: 0.7499 - acc: 0.6169 - val_loss: 1.4117 - val_acc: 0.5031
Epoch 64/100
11520/11520 [==============================] - 135s 12ms/step - loss: 0.7362 - acc: 0.6249 - val_loss: 1.8832 - val_acc: 0.3996
Epoch 65/100
11520/11520 [==============================] - 135s 12ms/step - loss: 0.7508 - acc: 0.6219 - val_loss: 1.3500 - val_acc: 0.5201
Epoch 66/100


In [11]:
### Dropout for loop
import json

# dp_list = [0.0,0.1,0.3,0.5]
batch_size = 80
epochs = 100
opt = keras.optimizers.Adam(lr=5*1e-4)
history_ls = []
dropout = 0.5
model3_drop = init_model3(bidir_mode='ave',cnn_dropout=dropout,rnn_dropout=dropout)
model3_drop.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
history = model3_drop.fit(x_train[::5], y_train[::5],
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)
# history_ls.append(history)
filename = 'history_do_5.json'
with open(filename, 'w') as outfile:
    json.dump(history.history, outfile)

Train on 11520 samples, validate on 14400 samples
Epoch 1/100
11520/11520 [==============================] - 138s 12ms/step - loss: 1.5658 - acc: 0.2775 - val_loss: 1.4973 - val_acc: 0.3506
Epoch 2/100
11520/11520 [==============================] - 135s 12ms/step - loss: 1.3192 - acc: 0.3662 - val_loss: 1.3772 - val_acc: 0.3340
Epoch 3/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.4156 - acc: 0.3570 - val_loss: 1.2880 - val_acc: 0.3681
Epoch 4/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.3875 - acc: 0.3733 - val_loss: 1.3919 - val_acc: 0.3261
Epoch 5/100
11520/11520 [==============================] - 135s 12ms/step - loss: 1.1722 - acc: 0.4451 - val_loss: 1.3519 - val_acc: 0.3381
Epoch 6/100
11520/11520 [==============================] - 134s 12ms/step - loss: 1.0482 - acc: 0.4793 - val_loss: 1.2304 - val_acc: 0.4571
Epoch 7/100
11520/11520 [==============================] - 135s 12ms/step - loss: 1.0105 - acc: 0.4983 - val_l

Epoch 59/100
11520/11520 [==============================] - 134s 12ms/step - loss: 0.7689 - acc: 0.5985 - val_loss: 2.1380 - val_acc: 0.3779
Epoch 60/100
11520/11520 [==============================] - 134s 12ms/step - loss: 0.7669 - acc: 0.5960 - val_loss: 1.8642 - val_acc: 0.4105
Epoch 61/100
11520/11520 [==============================] - 134s 12ms/step - loss: 0.7576 - acc: 0.6037 - val_loss: 2.4417 - val_acc: 0.3701
Epoch 62/100
11520/11520 [==============================] - 134s 12ms/step - loss: 0.7614 - acc: 0.6035 - val_loss: 2.3277 - val_acc: 0.3903
Epoch 63/100
11520/11520 [==============================] - 134s 12ms/step - loss: 0.7541 - acc: 0.6092 - val_loss: 1.9161 - val_acc: 0.4469
Epoch 64/100
11520/11520 [==============================] - 134s 12ms/step - loss: 0.7626 - acc: 0.6084 - val_loss: 2.1707 - val_acc: 0.4197
Epoch 65/100
11520/11520 [==============================] - 134s 12ms/step - loss: 0.7485 - acc: 0.6111 - val_loss: 2.3373 - val_acc: 0.4067
Epoch 66/100


In [ ]:
### Dropout for loop
import json


dp_list = [0.0,0.1,0.3,0.5]
batch_size = 80
epochs = 100
opt = keras.optimizers.Adam(lr=5*1e-4)
history_ls = []

for i,dropout in enumerate(dp_list):
    print('====================================')
    print('Dropout = ',dropout)
    model3_drop = init_model3(bidir_mode='ave',cnn_dropout=dropout,rnn_dropout=dropout)
    model3_drop.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
    history = model3_drop.fit(x_train[::5], y_train[::5],
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    history_ls.append(history)
    filename = 'history_do_' + str(dp_list[i])[-1] + '.json'
    with open(filename, 'w') as outfile:
        json.dump(history.history, outfile)

Dropout =  0.0
Train on 11520 samples, validate on 14400 samples
Epoch 1/100
11520/11520 [==============================] - 117s 10ms/step - loss: 1.6722 - acc: 0.2574 - val_loss: 1.5348 - val_acc: 0.3190
Epoch 2/100
11520/11520 [==============================] - 115s 10ms/step - loss: 1.5804 - acc: 0.3080 - val_loss: 1.5140 - val_acc: 0.3488
Epoch 3/100
11520/11520 [==============================] - 115s 10ms/step - loss: 1.4563 - acc: 0.3628 - val_loss: 1.7747 - val_acc: 0.3435
Epoch 4/100
11520/11520 [==============================] - 115s 10ms/step - loss: 1.5706 - acc: 0.3094 - val_loss: 1.3109 - val_acc: 0.3837
Epoch 5/100
11520/11520 [==============================] - 115s 10ms/step - loss: 1.6531 - acc: 0.2902 - val_loss: 1.6664 - val_acc: 0.2925
Epoch 6/100
11520/11520 [==============================] - 115s 10ms/step - loss: 1.5369 - acc: 0.3314 - val_loss: 1.7488 - val_acc: 0.1837
Epoch 7/100
11520/11520 [==============================] - 115s 10ms/step - loss: 1.3320 - acc:

11520/11520 [==============================] - 132s 11ms/step - loss: 0.7715 - acc: 0.6478 - val_loss: 0.8074 - val_acc: 0.6353
Epoch 16/100
11520/11520 [==============================] - 133s 12ms/step - loss: 0.7441 - acc: 0.6592 - val_loss: 0.9567 - val_acc: 0.5958
Epoch 17/100
11520/11520 [==============================] - 132s 11ms/step - loss: 0.7241 - acc: 0.6716 - val_loss: 0.7500 - val_acc: 0.6649
Epoch 18/100
11520/11520 [==============================] - 133s 12ms/step - loss: 0.7253 - acc: 0.6720 - val_loss: 0.7790 - val_acc: 0.6452
Epoch 19/100
11520/11520 [==============================] - 133s 12ms/step - loss: 0.6991 - acc: 0.6825 - val_loss: 0.7845 - val_acc: 0.6492
Epoch 20/100
11520/11520 [==============================] - 132s 11ms/step - loss: 0.6822 - acc: 0.6925 - val_loss: 0.7634 - val_acc: 0.6559
Epoch 21/100
11520/11520 [==============================] - 133s 12ms/step - loss: 0.6678 - acc: 0.6943 - val_loss: 0.8073 - val_acc: 0.6398
Epoch 22/100
11520/11520 [

11520/11520 [==============================] - 133s 12ms/step - loss: 1.7917 - acc: 0.1674 - val_loss: 1.7920 - val_acc: 0.1667
Epoch 31/100
11520/11520 [==============================] - 133s 12ms/step - loss: 1.7917 - acc: 0.1670 - val_loss: 1.7919 - val_acc: 0.1667
Epoch 32/100
11520/11520 [==============================] - 133s 12ms/step - loss: 1.7917 - acc: 0.1684 - val_loss: 1.7920 - val_acc: 0.1667
Epoch 33/100
11520/11520 [==============================] - 133s 12ms/step - loss: 1.7917 - acc: 0.1676 - val_loss: 1.7919 - val_acc: 0.1667
Epoch 34/100
11520/11520 [==============================] - 133s 12ms/step - loss: 1.7917 - acc: 0.1701 - val_loss: 1.7919 - val_acc: 0.1667
Epoch 35/100
11520/11520 [==============================] - 133s 12ms/step - loss: 1.7917 - acc: 0.1701 - val_loss: 1.7920 - val_acc: 0.1667
Epoch 36/100
11520/11520 [==============================] - 133s 12ms/step - loss: 1.7917 - acc: 0.1670 - val_loss: 1.7919 - val_acc: 0.1667
Epoch 37/100
11520/11520 [

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt

for i,history in enumerate(history_ls):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()